In [2]:
import typing
from typing import List
from midiutil import MIDIFile
Chord = List[int]
QuarterNote = List[Chord | 'hold']

def convert_note_group_sequence_to_midi(left_hand: List[QuarterNote], right_hand: List[QuarterNote], save_path):
  right_track = 0
  left_track = 1
  channel = 0
  tempo = 100
  volume = 100
  midi_file = MIDIFile(2)
  midi_file.addTempo(right_track, 0, tempo)
  
  for hand in [right_hand, left_hand]:
    if hand == right_hand:
      track, octave, volume = right_track, 4, 100
    else:
      track, octave, volume = left_track, 3, 75
    for time, quarter in enumerate(hand):
      note_duration = 1 / len(quarter)
      for i, note in enumerate(quarter):
        if note == 'hold':
          continue
        note_time = time + note_duration * i
        print(note)
        for pitch in note:
          midi_file.addNote(track, channel, pitch + 12 * octave, note_time, note_duration, volume)
  
  with open(save_path, 'wb') as output_file:
    midi_file.writeFile(output_file)
    

In [3]:

def rotate_left(items: List[typing.Any], n: int) -> List[typing.Any]:
  return items[n % len(items):] + items[:n % len(items)]


class Key:
  _accidentals = {'♮': 0, 'b': -1, '#': 1}
  def __init__(self, pitches: List[int]):
    self._pitches = pitches
    
  @classmethod
  def major(cls):
    return Key([0, 2, 4, 5, 7, 9, 11])
  
  @classmethod
  def minor(cls):
    return Key([0, 2, 3, 5, 7, 8, 10])
  
  @classmethod
  def diminished(cls):
    return Key([0, 2, 3, 5, 6, 8, 9, 11])
  
  @classmethod
  def augmented(cls):
    return Key([0, 3, 4, 7, 8, 11])
    
  @property
  def pitches(self) -> List[int]:
    return self._pitches
  
  def __getitem__(self, index: int | str) -> 'Key':
    # TODO: this does not currently respect octave
    if isinstance(index, str):
      offset = Key._accidentals[index[-1]]
      index = int(index[:-1])
    else:
      offset = 0
    index -= 1
    return Key([pitch + offset for pitch in rotate_left(self.pitches, index)])
  
  def pitch(self, index: int | str) -> int:
    if isinstance(index, str):
      offset = Key._accidentals[index[-1]]
      index = int(index[:-1])
    else:
      offset = 0
    index -= 1
    relative_index = index % len(self.pitches)
    octave = index // len(self.pitches)
    return self.pitches[relative_index] + octave * 12 + offset
  
  def chord(self, notes: List[int], required: List[int] = None, cardinality: int = None, inversion: int = 1, exclusion_preference: List[int] = None):
    if required is None:
      required = notes
    if cardinality is None:
      cardinality = len(notes)
    if exclusion_preference is None:
      exclusion_preference = []
      
    included_notes = set(required)
    remaining_notes = set(notes) - included_notes
    remaining_preferred_notes = remaining_notes - set(exclusion_preference)
    remaining_un_preferred_notes = remaining_notes - remaining_preferred_notes
    
    while len(included_notes) < cardinality and (remaining_preferred_notes or remaining_un_preferred_notes):
      if remaining_preferred_notes:
        included_notes.add(remaining_preferred_notes.pop())
      else:
        included_notes.add(remaining_un_preferred_notes.pop())
    # TODO: this does not properly do inversions respecting octave
    return rotate_left([self.pitch(note) for note in notes], inversion - 1)
  
  def as_root(self):
    return self[-min(self.pitches)]
  
  def is_major_like(self):
    # check that note 3 is a major third from the base note
    return self.as_root().pitch(3) == 4
  
  def is_minor_like(self):
    # check that note 3 is flatted
    return self.as_root().pitch(3) == 3
  
  def is_diminished_like(self):
    # check that notes 3 and 5 are flatted
    as_root = self.as_root()
    return as_root.pitch(3) == 3 and as_root.pitch(5) == 6
  
  def is_augmented_like(self):
    # check that note 5 is sharped
    return self.as_root().pitch(5) == 8
  
  def __str__(self) -> str:
    return ' '.join([str(pitch) for pitch in self.pitches])

In [18]:
from dataclasses import dataclass
import difflib
import math
import random

# class Duration:
#   def __init__(self, beats: float):
#     self._subunits = int(beats * 4 * 3)
#     
#   def __eq__(self, other):
#     return isinstance(other, Duration) and self._subunits == other._subunits
#   
#   def __lt__(self, other):
#     return isinstance(other, Duration) and self._subunits < other._subunits
#   
#   def __gt__(self, other):
#     return isinstance(other, Duration) and self._subunits > other._subunits


@dataclass
class Note:
  pitch: int
  duration: float
  
  
@dataclass
class Chord:
  key: Key
  notes: List[int]
  duration: float
  required: List[int] = None
  cardinality: int = None
  inversion: int = 1
  exclusion_preference: List[int] = None
  
  
class NoteSequence:
  def __init__(self, notes: List[Note] = None):
    self.notes = notes or []
    
  def append(self, note: Note) -> None:
    self.notes.append(note)
    
  def __add__(self, other):
    return NoteSequence(self.notes + other.notes)
  
  def __getitem__(self, item: int) -> Note:
    return self.notes[item]
  
  def __len__(self) -> int:
    return len(self.notes)
  
  @property
  def pitches(self) -> List[int]:
    return [note.pitch for note in self]
  
  @property
  def rhythm(self) -> List[float]:
    return [note.duration for note in self]
  
  @property
  def duration(self) -> float:
    return sum(self.rhythm)


def compare_pitch_order(s: NoteSequence, pitches: List[int]) -> int:
  # just get sequences of unique pitches
  p1 = []
  for p in s.pitches:
    if not p1 or p1[-1] != p:
      p1.append(p)
  p2 = []
  for p in pitches:
    if not p2 or p2[-1] != p:
      p2.append(p)
  matcher = difflib.SequenceMatcher(None, p1, p2)
  lcs = sum(block.size for block in matcher.get_matching_blocks())
  return lcs

def compare_shape(s: NoteSequence, pitches: List[int]) -> int:
  # just get sequences of unique pitches
  p1 = []
  for p in s.pitches:
    if not p1 or p1[-1] != p:
      p1.append(p)
  p2 = []
  for p in pitches:
    if not p2 or p2[-1] != p:
      p2.append(p)
  matcher = difflib.SequenceMatcher(None, p1, p2)
  lcs = sum(block.size for block in matcher.get_matching_blocks())
  return lcs

def compare_rhythm(s: NoteSequence, durations: List[float]) -> int:
  num_sync_note_beginnings = 0
  
  target_duration = 0
  sequence_duration = 0
  sequence_index = 0
  for duration in durations:
    target_duration += duration
    while sequence_duration < target_duration and sequence_index < len(s):
      sequence_duration += s[sequence_index].duration
      sequence_index += 1
    if sequence_duration == target_duration:
      num_sync_note_beginnings += 1
      
  sync_note_ratio = num_sync_note_beginnings / len(durations)
  note_count_difference_ratio = abs(len(s) - len(durations)) / len(durations)
  return int((sync_note_ratio * 2 - note_count_difference_ratio) * len(durations) * 2)
  
def compare_pitch_set(s: NoteSequence, pitches: List[int]) -> int:
  return len(set(pitches) & set(s.pitches))
  

def best_note_sequences(
        note_sequences: List[NoteSequence],
        target_pitches: List[int] = None,
        target_rhythm: List[float] = None
) -> NoteSequence:
  indices = range(len(note_sequences))
  scores = [0] * len(indices)
  if target_pitches is not None:
    random.shuffle(indices)
    ranked_by_pitch_order = sorted(indices, key=lambda i: compare_pitch_order(note_sequences[i], target_pitches))
    random.shuffle(indices)
    ranked_by_pitch_set = sorted(indices, key=lambda i: compare_pitch_set(note_sequences[i], target_pitches))
    for pitch_order_rank, i in enumerate(ranked_by_pitch_order):
      scores[i] += pitch_order_rank
    for pitch_set_rank, i in enumerate(ranked_by_pitch_set):
      scores[i] += pitch_set_rank / 2  # not as important as pitch order or rhythm
  if target_rhythm is not None:
    random.shuffle(indices)
    ranked_by_rhythm = sorted(indices, key=lambda i: compare_rhythm(note_sequences[i], target_rhythm))
    for rhythm_rank, i in enumerate(ranked_by_rhythm):
      scores[i] += rhythm_rank
  return note_sequences[max([(index, score) for index, score in enumerate(scores)], key=lambda x: x[1])[0]]
  
  
def generate_note_sequence(
        start_pitch: int,
        end_pitch: int,
        allowed_note_durations: List[float],
        target_duration: float,
        possible_continuations: typing.Callable[[NoteSequence, int, float], typing.Set[NoteSequence]],
        include_end_pitch: bool,
        target_pitches: List[int] = None,
        target_rhythm: List[float] = None,
        max_notes: int = None
) -> NoteSequence:
  if max_notes is None:
    max_notes = target_duration * 4
  possible_sequences = []
  heads = set([NoteSequence([Note(start_pitch, duration)]) for duration in allowed_note_durations])
  while heads:
    head = heads.pop()
    for continuation in possible_continuations(head, end_pitch, target_duration):
      sequence = head + continuation
      if len(sequence) > (max_notes if include_end_pitch else max_notes + 1):
        continue
      sequence_duration = sequence.duration if include_end_pitch else sequence.duration - sequence[-1].duration
      if sequence_duration == target_duration and sequence[-1].pitch == end_pitch:
        possible_sequences.append(sequence if include_end_pitch else NoteSequence(sequence[:-1]))
      elif sequence < target_duration:
        heads.add(sequence)
  return best_note_sequences(possible_sequences, target_pitches, target_rhythm)
# modify this to use the new note classes and stuff
# make the continuations use the rhythm function to generate 

In [17]:
def arpeggio_continuations(chord_progression: List[Chord]):
  # find the most recent peak and trough and do for both all downturns and upturns
  def function(sequence: NoteSequence, end_pitch: int, target_duration: float):
    pass
  
class Arpeggio:
  

6